# 한글문서의 텍스트마이닝 : 감성분석
1. 데이터 준비 및 탐색
    1. 데이터 로딩
    1. 데이터 탐색 및 정제
1. 분석 모델 구축
    1. Feature Vector 생성
    1. 감성 분석 모델 구축
1. 분석 모델 평가
    1. 평가 데이터로 정확도 확인
    1. 새로운 텍스트에 대한 감성 예측

## 1. 데이터 준비 및 탐색
1. 데이터 로딩
1. 데이터 정보 확인 및 정제

### 1-1. 데이터 로딩
* 훈련용 데이터(ratings_train.txt)
* 평가용 데이터(ratings_test.txt)

In [2]:
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep="\t", encoding="utf8")
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [3]:
test_df = pd.read_csv("ratings_test.txt", sep="\t", encoding="utf8")
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


## 1-2. 데이터의 정보 확인 및 정제
### 1-2-1. 결측치 확인 및 처리

In [4]:
#결측치 확인해보고
train_df.info()

train_df.dropna(inplace=True) #결측치는 삭제로 처리
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [5]:
test_df.info()

test_df.dropna(inplace=True) #결측치는 삭제로 처리
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


### 1-2-2. 타겟 컬럼 label 확인 (0: 부정감성,   1: 긍정감성)

In [6]:
train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [7]:
test_df['label'].value_counts()

1    25171
0    24826
Name: label, dtype: int64

### 1-2-3. 한글 이외의 문자는 공백으로 변환 (정규표현식 이용)

In [8]:
import re


#^ : not, +는 연속된 문자
sub_func = lambda x : re.sub(r'[^ㄱ-ㅣ가-힣\s]+', ' ', x)
train_df['document'].apply(sub_func)

0                                        아 더빙  진짜 짜증나네요 목소리
1                              흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                               교도소 이야기구먼  솔직히 재미는 없다 평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                    인간이 문제지  소는 뭔죄인가 
149996                                          평점이 너무 낮아서 
149997                      이게 뭐요  한국인은 거들먹거리고 필리핀 혼혈은 착하다 
149998                          청춘 영화의 최고봉 방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [9]:
test_df['document'].apply(sub_func)

0                                                  굳 ㅋ
1                                                     
2                   뭐야 이 평점들은  나쁘진 않지만  점 짜리는 더더욱 아니잖아
3                          지루하지는 않은데 완전 막장임  돈주고 보기에는 
4         만 아니었어도 별 다섯 개 줬을텐데  왜  로 나와서 제 심기를 불편하게 하죠 
                             ...                      
49995        오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996        의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따  
49997                 그림도 좋고 완성도도 높았지만  보는 내내 불안하게 만든다
49998     절대 봐서는 안 될 영화  재미도 없고 기분만 잡치고  한 세트장에서 다 해먹네
49999                                       마무리는 또 왜이래
Name: document, Length: 49997, dtype: object

# 2. 분석 모델 구축
1. Feature Vector 생성
1. 감성 분석 모델 구축

## 2-1. 피처 벡터화 : TF-IDF
1. 형태소를 분석하여 토근화 : konlpy.Okt 라이브러리 이용
1. TF-IDF 기반의 feature vector 생성

### 2-1-1. 형태소를 분석하여 토큰화 함수 구현
* koNLPy.Okt 이용 : *tokens* = *OktObject*.**morphs**(*text*)

In [10]:
from konlpy.tag import Okt
okt = Okt()

def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

### 2-1-2. TF-IDF 기반 피처 벡터 생성 
* sklearn 라이브러리의 TfidfVectorizer로 학습하여 Feature Vectore 생성 모델 구축
   - from sklearn.feature_extraction.text import TfidfVectorizer
   - 모델 생성 : *model* = **TfidfVectorizer**(tokenizer=*tokenizer_func*, *optional parameters*)
   - 모델 학습 : *model*.**fit**(*train_text_data*)
   - Feature Vertor로 변환 : *feature_vector* = *model*.**transform**(*text_data*)
* 실행시간 10분 정도 걸림

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 모델 생성
tfidf_model = TfidfVectorizer(tokenizer=okt_tokenizer)

#모델 학습
tfidf_model.fit(train_df['document'])

# text -> feature vector
train_tfidf = tfidf_model.transform(train_df['document'])


C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## 2-2) 감성 분류(Sentiment Analysis) 모델 구축

* 로지스틱 회귀를 이용한 이진 분류

1. 로지스틱 회귀 기반 분석 모델 학습 (sklearn.linear_model.LogisticRegression)
1. 최적의 하이퍼파라미터 갖는 학습 모델 구축 (sklearn.model_selection.**GridSearchCV**)
1. 최적 학습 모델에 대한 평가

### 2-2-1. 로지스틱 회귀 기반 분석 모델 학습

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [12]:
from sklearn.linear_model import LogisticRegression
# 모델 생성
model = LogisticRegression()

#모델 학습
X_train = train_tfidf
y_train = train_df['label']
model.fit(X_train, y_train)

# text -> feature vector
train_score = round(model.score(X_train, y_train)*100, 2)
train_score

C:\Users\User\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


89.43

### 2-2-2. 로지스틱 회귀의  최적의 하이퍼파라미터 찾기

* 로지스틱 회귀의 하이퍼파라미터 C의 최적값 찾기

1. 최적의 하이퍼파라미터 찾는 객체 생성 : *gridSearchCVobject* = **GridSearchCV**(*estimater*, *param_grid*, *optional parameters*)
1. 학습 데이터로 최적의 하이퍼파라미터 도출 : *gridSearchCVobject*.**fit**(*train_data*)
1. 최적의 하이퍼파라미터를 적용한 학습 모델 저장 : *best_model* = *gridSearchCVobject*.**best_estimator_**

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

### 2-2-3. 최적 분석 모델 저장

# 3. 분석 모델 평가

## 3-1. 평가용 데이터를 이용하여 감성 분석 모델 정확도

In [13]:
# 평가용 데이터의 피처 벡터화 : 실행시간 6분 정도 걸립니다 ☺
test_tfidf = tfidf_model.transform(test_df['document'])

In [14]:
#모델 평가
test_score = round(model.score(test_tfidf, test_df['label'])*100, 2)
test_score

85.13

In [15]:
SA_LogR_model = model

## 3-2. 새로운 텍스트에 대한 감성 예측
1. 문장 입력 받기
2. 입력 문장 전처리
3. Feature Vector 생성
4. 감성 분석 모델 적용 -> 감성 분석
5. 예측값 출력

In [16]:
#문장 입력 받기
in_sent = input("감정을 분석할 문장을 입력하세요 >>")

#입력 문장 전처리
cl_sent = sub_func(in_sent)

#Feature Vector 생성
in_list = []
in_list.append(cl_sent)
st_tfidf = tfidf_model.transform(in_list)

#감성 분석 모델 적용 -> 감성 분석
st_predict = SA_LogR_model.predict(st_tfidf)

#예측값 출력
print(in_sent, ': 긍정감성' if st_predict else print(in_sent, ':부정감성'))

이하리 졸귀탱 : 긍정감성


# 4. Crawling 데이터에 대한 SA

# 여기부터~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [17]:
# Crawling 데이터 로딩하기
import pandas as pd
filename = "강아지_news.json"
temp_df = pd.read_json(filename)
news_df = temp_df[['title', 'description']]
news_df

,title,description
0,"[팝업★]'나솔' 16기 영숙, 돌연 前남편 공개 &quot;찢어 죽이고 싶은 사람...",공개된 사진 속에는 글로 언급했던 <b>강아지</b> 모양의 케이크부터 토끼 솜사탕...
1,[미래인재를 키우는 충남교육 참학력] AI와 함께 미래로!… 천안교육지원청,관내 모든 초·중학생들에게 교과·창의적 체험활동 연계한 AI교육 프로그램을 제공하고...
2,"오늘의집, 세계 동물의 날 기념 선물캠페인 '코코야 고마워' 진행","오늘의집은 고객이 필요한 상품을 쉽게 찾도록 <b>강아지</b>, 고양이 별로 ▲리..."
3,"웨이브, 오리지널 '거래'·'악인취재기' 등 10월 라인업 공개","◇ '연인 파트2', '오늘도 사랑스럽개' 등 뉴 페이스 등장 남궁민과 안은진의 애..."
4,김유리 &quot;중학생 때 시력 잃고 7년 동안 숨어 살아…희망 주는 가수 될 것...,"또 “당시 내겐 엄마와 외할머니, <b>강아지</b>뿐이었다”며 “엄마는 마음 아파..."
...,...,...
95,[기획] 제28회 부산국제영화제 추천작 ①,"역사가였던 어머니부터 시작해 이웃집 <b>강아지</b>, 여러 편의 영화를 촬영했던..."
96,주인 다리 물어뜯은 반려견이 안락사 대신 '영웅견'이라 불리게 된 사연,= 주인의 다리를 물어뜯어 피까지 나게 한 <b>강아지</b> 한 마리가 오히려 ‘...
97,"윤혜진, ♥엄태웅 복귀 앞두고 행복한 부산 여행",이어 공개된 사진에서 엄태웅 윤혜진 부부는 딸 지온이와 <b>강아지</b> 봉구까지...
98,"`톡파원 25시`, 이찬원의 상냥한 눈동자는 눈앞의 사람을 미소 짓게 하고 행복...","한편, 오는 10월 9일 월요일 오후 8시 50분 JTBC `톡파원 25시`에서는 ..."


In [18]:
# 입력 문장 전처리
import re

sub_func = lambda x : re.sub(r'[^ㄱ-ㅣ가-힣\s]+', ' ', x)
news_df['title'] = news_df['title'].apply(sub_func)
news_df['description'] = news_df['description'].apply(sub_func)
news_df

C:\Users\User\AppData\Local\Temp\ipykernel_13652\4034741373.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['title'] = news_df['title'].apply(sub_func)
C:\Users\User\AppData\Local\Temp\ipykernel_13652\4034741373.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['description'] = news_df['description'].apply(sub_func)


,title,description
0,팝업 나솔 기 영숙 돌연 남편 공개 찢어 죽이고 싶은 사람이었지만,공개된 사진 속에는 글로 언급했던 강아지 모양의 케이크부터 토끼 솜사탕 그리고...
1,미래인재를 키우는 충남교육 참학력 와 함께 미래로 천안교육지원청,관내 모든 초 중학생들에게 교과 창의적 체험활동 연계한 교육 프로그램을 제공하고 ...
2,오늘의집 세계 동물의 날 기념 선물캠페인 코코야 고마워 진행,오늘의집은 고객이 필요한 상품을 쉽게 찾도록 강아지 고양이 별로 리빙 푸드 ...
3,웨이브 오리지널 거래 악인취재기 등 월 라인업 공개,연인 파트 오늘도 사랑스럽개 등 뉴 페이스 등장 남궁민과 안은진의 애절한...
4,김유리 중학생 때 시력 잃고 년 동안 숨어 살아 희망 주는 가수 될 것,또 당시 내겐 엄마와 외할머니 강아지 뿐이었다 며 엄마는 마음 아파했지만 묵...
...,...,...
95,기획 제 회 부산국제영화제 추천작,역사가였던 어머니부터 시작해 이웃집 강아지 여러 편의 영화를 촬영했던 자신의 집...
96,주인 다리 물어뜯은 반려견이 안락사 대신 영웅견 이라 불리게 된 사연,주인의 다리를 물어뜯어 피까지 나게 한 강아지 한 마리가 오히려 영웅견 ...
97,윤혜진 엄태웅 복귀 앞두고 행복한 부산 여행,이어 공개된 사진에서 엄태웅 윤혜진 부부는 딸 지온이와 강아지 봉구까지 함께 다...
98,톡파원 시 이찬원의 상냥한 눈동자는 눈앞의 사람을 미소 짓게 하고 행복,한편 오는 월 일 월요일 오후 시 분 톡파원 시 에서는 끓어오르는 ...


In [88]:
# Feature Vector 생성
news_title_tfidf = tfidf_model.transform(news_df.loc[:, 'title'])
news_desc_tfidf = tfidf_model.transform(news_df.loc[:, 'description'])

# 감성 분석 모델 적용 -> 감성 분석
news_title_predict = SA_LogR_model.predict(news_title_tfidf)
news_desc_predict = SA_LogR_model.predict(news_desc_tfidf)

# 예측값 출력
news_df.loc[:, 'title_SA'] = news_title_predict
news_df.loc[:, 'description_SA'] = news_desc_predict

#확인용
news_df.loc[:, 'title_SA'] # 시리즈
news_df.loc[:, 'title_SA'].values # 배열(s 주의)
news_df.loc[:, 'title_SA'].value_counts() #값별 개수(counts만 s있음에 주의)

C:\Users\User\AppData\Local\Temp\ipykernel_5268\349219203.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.loc[:, 'title_SA'] = news_title_predict
C:\Users\User\AppData\Local\Temp\ipykernel_5268\349219203.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df.loc[:, 'description_SA'] = news_desc_predict


1    51
0    49
Name: title_SA, dtype: int64

In [32]:
# 해당 키워드에 대한 긍정 뉴스와 부정 뉴스의 비율 출력하기
rate_good_title = round((news_df['title_SA'].value_counts()[0]/len(news_df['title_SA']))*100, 2)
rate_bad_title = round((news_df['title_SA'].value_counts()[1]/len(news_df['title_SA']))*100, 2)

print(f'[제목]긍정적 : {rate_good_title}%')
print(f'[제목]부정적 : {rate_bad_title}%')

rate_good_desc = round((news_df['description_SA'].value_counts()[0]/len(news_df['description_SA']))*100, 2)
rate_bad_desc = round((news_df['description_SA'].value_counts()[0]/len(news_df['description_SA']))*100, 2)

print(f'[내용]긍정적 : {rate_good_desc}%')
print(f'[내용]부정적 : {rate_bad_desc}%')


[제목]긍정적 : 49.0%
[제목]부정적 : 51.0%
[내용]긍정적 : 50.0%
[내용]부정적 : 50.0%
